![title](../assets/problem.png)

In [ ]:
import json
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import plotly.express as px
from typing import Dict, List, Union, Any
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 5000)
pd.set_option('max_colwidth', 5000)

In [ ]:
BASE_PATH = "/Users/seanariel/Desktop/la-maniee/data/mlops"

PATH_TO_SYNTHETIC_DATA = f"{BASE_PATH}/synthetic_data_contract.csv"
PATH_TO_EXPLODED_FEATURES = f"{BASE_PATH}/exploded_features.csv"
PATH_TO_FEATURE_STORE = f"{BASE_PATH}/feature_store.csv"
PATH_TO_DEV_TRAINING_DATA = f"{BASE_PATH}/dev_training.csv"
PATH_TO_DEV_TESTING_DATA = f"{BASE_PATH}/dev_testing.csv"
PATH_TO_AUTOML_TRAINING_DATA = f"{BASE_PATH}/automl_training.csv"
PATH_TO_PRECISION_RECALL = f"{BASE_PATH}/precision_recall.csv"
PATH_TO_OPTIMAL_MODEL = f"{BASE_PATH}/optimal_model.pickle"
PATH_TO_PRODUCTION_MODEL = f"{BASE_PATH}/production_model.pickle"
PATH_TO_TRAINING_DATA = f"{BASE_PATH}/training.csv"
PATH_TO_EXPERIMENTATION_DATA = f"{BASE_PATH}/experimentation.csv"

# Table of Content:
* [Overview](#first-bullet)
* [Feature Engineering](#second-bullet)
* [Model Development](#third-bullet)
* [Model Training](#fourth-bullet)
* [Model Serving](#fifth-bullet)
* [Model Experimentation](#sixth-bullet)

# Feature Engineering <a class="anchor" id="second-bullet"></a>

In [ ]:
FEATURE_CONVERTER_ENDPOINT = "http://34.77.247.189:1337/cards_to_features"


def clean_feature(hand: str, return_as_scalar: bool = True) -> Dict[str, int]:
    """
    In:
    {
        "raw_hand": "8H.KC.QH.9D.QC.TC.7H.QD.AH.JD.TH.TS"
    }
    Out:
        has_3_cards_in_suit_clubs
        has_2_cards_in_suit_diamonds
        has_5_cards_in_suit_hearts
        has_1_cards_in_suit_spades
        [...]
    """
    response = requests.post(
        FEATURE_CONVERTER_ENDPOINT, 
        json={"raw_hand": hand, "return_as_scalar": return_as_scalar}
    )
    features: Dict = response.json()
    return features

# Model Serving <a class="anchor" id="fifth-bullet"></a>

In [ ]:
import pickle
from werkzeug.wrappers import Request, Response
from flask import Flask, jsonify, request
import requests
import pandas as pd

In [ ]:
SAMPLE = 1
feature_store = pd.read_csv(PATH_TO_FEATURE_STORE, nrows=SAMPLE)

In [ ]:
POTENTIAL_TARGETS = ["reward", "p1_has_won"]
TARGET = "p1_has_won"
SEGMENTS = ["reward", "contract"]
COVARIATES = list(filter(lambda covariate: covariate not in (POTENTIAL_TARGETS + SEGMENTS), feature_store.columns))

### Prepare the features

In [ ]:
"""
The convention here to follow is to input the hand correctly to the API endpoint.

Names: 7, 8, 9, T(=10), J(=jack), Q(=queen), K(=king), A(=ace)
Suits: C(=clubs), D(=diamonds), H(=hearts), S(=spades)

Hand Inputing:

-----------------------
     8D KC JS 7S (=> to compute p2_face_value)
     
     9C JC KH 8S
AS QS TS 9S TD AD KD TC

-----------------------
      8 9 10 11
   0 1 2 3 4 5 6 7
-----------------------

=> AS.QS.TS.9S.TD.AD.KD.TC.9C.JC.KH.8S
"""

HAND = "AS.QS.TS.9S.TD.AD.KD.TC.9C.JC.KH.8S"

In [ ]:
data = {
    "raw_features": HAND,
    "last_bidder": 1, 
    "starter": 1,
    "clubs": 0,
    "diamonds": 0,
    "hearts": 0,
    "spades": 1,
    "sans_atouts": 0,
    "tout_atouts": 0,
    "p1_face_value": 79,
    "p2_face_value": 24
}
data

In [ ]:
"""
To be able to leverage the model, we must replicate the feature engineering pipeline,
which means building the same features.

This code replication is very bad practices as we are implementing the same functionality in 2 different pipelines, 
so we must now maintain both as code changes
"""
def build_extra_features(features_dict: Dict) -> Dict:
    features_dict["total_BR_points"] = (
    features_dict["has_BR_at_clubs"] +
    features_dict["has_BR_at_diamonds"] +
    features_dict["has_BR_at_hearts"] +
    features_dict["has_BR_at_spades"]
    )
    features_dict["total_tierce_points"] = (
        features_dict["has_tierce_at_clubs"] +
        features_dict["has_tierce_at_diamonds"] +
        features_dict["has_tierce_at_hearts"] +
        features_dict["has_tierce_at_spades"]
    )
    features_dict["total_AnD_points"] = features_dict["total_BR_points"] + features_dict["total_tierce_points"]
    return features_dict

### Launch the webserver

In [ ]:
app = Flask(__name__)

@app.route("/serve_model", methods=["POST"])
def serve_model():
    """
    In:
    {
        'raw_features': 'AS.QS.TS.9S.TD.AD.KD.TC.9C.JC.KH.8S',
        "last_bidder": 1,
        "starter": 1,
        "clubs": 0,
        "diamonds": 0,
        "hearts": 0,
        "spades": 1,
        "sans_atouts": 0,
        "tout_atouts": 0,
        "p1_face_value": 79,
        "p2_face_value": 24
    }
    Out:
    {
        "prediction": {
            "win_probability": [
                0.61
            ]
        }
    }
    """
    data: Dict = request.json
    raw_features: str = data.get("raw_features")
    other_info: Dict = {k: v for k, v in data.items() if k != "raw_features"}
    processed_features = clean_feature(raw_features, return_as_scalar=True)
    processed_features = build_extra_features(processed_features)
    processed_features = {**other_info, **processed_features}
    processed_features = {k: [v] for k, v in processed_features.items()}
    features_df = pd.DataFrame()
    features_df = pd.concat([features_df, pd.DataFrame(processed_features)])
    with open(PATH_TO_PRODUCTION_MODEL, 'rb') as handle:
        production_model = pickle.load(handle)
    predictions = production_model.predict_proba(features_df[COVARIATES])[:, 1]
    return jsonify({"prediction": {"win_probability": predictions.tolist()}})

if __name__ == '__main__':
    
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

### [Optional] Assignment 5 - Google Cloud Functions

Let's deployed a serverless pipeline through <a> Google Cloud Functions </a>. This will allow us to serve our model at minimal cost and downtime.

But first, make sure to follow this lab on <a> POSTMAN </a>, which will be very useful to consume our model serving later on.

Once complete, head over to Google Cloud Functions to complete the lab

#### Credit

Note:
This content has been developed by Sean Ariel for educational purposes, specifically for Nuclio Digital School students. 
It is a practical training that cannot be copied, reproduced, distributed without the explicit consent from the author. © Sean Ariel